# Graph generation

This is the implementation of the paper "GraphRNN: Generating Realistic Graphs with Deep Auto-regressive Models".
The aim is to implement a neural network that is fed with graphs beforehand and then generates new ones.
An example application would be to take organic molecules as graphs and then create new potential compounds (You et al. 2018).
(You et al. 2018) .
.


An undirected graph $G = (V, E)$ is defined by its vertex set
$V = \{v_1 , \dots , v_n \} $ and the edge set $E \subset \{(v_i , v_j ) \mid v_i , v_j \in V \} $.
Furthermore, we want to define a node order $ \pi \colon V \to \mathbb{N} $ that assigns a natural number to each node and is an injective function.
Using the node order, we can represent each graph $G$ by an adjacency matrix $ A^\pi \in \mathbb{R}^{n \times n} $ where the $ (k, l) $ entry is equal to $ 1 $ if if $ \pi(v_i) = k $ and $ \pi(v_j) = l $, then $ (v_i, v_j) \in E $ holds, and otherwise the entry is $ 0 $. Let $ \Pi $ be the set of all node orders of the graph $ G $.

The goal of learning generative models of graphs is to learn
a distribution $p_{\text{model}} (\tilde{G}) $ over graphs based on a set of
observed graphs $\tilde{G} = \{G_1 , \dots, G_s \} $ coming from the known distribution $p$, where each graph $ G_i $ has a different finite number of nodes and edges.
number of nodes and edges.



In [ ]:
import networkx as nx
import numpy as np
import pygraphviz as pgv
from networkx.drawing.nx_agraph import graphviz_layout, to_agraph
from queue import Queue
from random import choice, randint

from torch import nn
import torch.optim as optim
import torch
import torch.nn.functional as F

from eval import stats
from torch.utils.tensorboard import SummaryWriter
import pdb
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
from copy import copy
#from data import *
import random
import matplotlib.pyplot as plt


: 

In [2]:
## creates data for validation, training and testing
## Quelle: https://github.com/snap-stanford/GraphRNN/blob/master/create_graphs.py
def create(args):
### load datasets
    graphs=[]
    # synthetic graphs
    if args=='ladder':
        graphs = []
        for i in range(100, 201):
            graphs.append(nx.ladder_graph(i))
        max_prev_node = 10
    elif args=='ladder_small':
        graphs = []
        for i in range(2, 11):
            graphs.append(nx.ladder_graph(i))
        max_prev_node = 10
    elif args=='tree':
        graphs = []
        for i in range(2,5):
            for j in range(3,5):
                graphs.append(nx.balanced_tree(i,j))
        max_prev_node = 256
    elif args=='caveman':
        # graphs = []
        # for i in range(5,10):
        #     for j in range(5,25):
        #         for k in range(5):
        #             graphs.append(nx.relaxed_caveman_graph(i, j, p=0.1))
        graphs = []
        for i in range(2, 3):
            for j in range(30, 81):
                for k in range(10):
                    graphs.append(caveman_special(i,j, p_edge=0.3))
        max_prev_node = 100
    elif args=='caveman_small':
        # graphs = []
        # for i in range(2,5):
        #     for j in range(2,6):
        #         for k in range(10):
        #             graphs.append(nx.relaxed_caveman_graph(i, j, p=0.1))
        graphs = []
        for i in range(2, 3):
            for j in range(6, 11):
                for k in range(20):
                    graphs.append(caveman_special(i, j, p_edge=0.8)) # default 0.8
        max_prev_node = 20
    elif args=='caveman_small_single':
        # graphs = []
        # for i in range(2,5):
        #     for j in range(2,6):
        #         for k in range(10):
        #             graphs.append(nx.relaxed_caveman_graph(i, j, p=0.1))
        graphs = []
        for i in range(2, 3):
            for j in range(8, 9):
                for k in range(100):
                    graphs.append(caveman_special(i, j, p_edge=0.5))
        max_prev_node = 20
    elif args.startswith('community'):
        num_communities = int(args[-1])
        print('Creating dataset with ', num_communities, ' communities')
        c_sizes = np.random.choice([12, 13, 14, 15, 16, 17], num_communities)
        #c_sizes = [15] * num_communities
        for k in range(3000):
            graphs.append(n_community(c_sizes, p_inter=0.01))
        max_prev_node = 80
    elif args=='grid':
        graphs = []
        for i in range(10,20):
            for j in range(10,20):
                graphs.append(nx.grid_2d_graph(i,j))
        max_prev_node = 40
    elif args=='grid_small':
        graphs = []
        for i in range(2,5):
            for j in range(2,6):
                graphs.append(nx.grid_2d_graph(i,j))
        max_prev_node = 15
    elif args=='barabasi':
        graphs = []
        for i in range(100,200):
             for j in range(4,5):
                 for k in range(5):
                    graphs.append(nx.barabasi_albert_graph(i,j))
        max_prev_node = 130
    elif args=='barabasi_small':
        graphs = []
        for i in range(4,21):
             for j in range(3,4):
                 for k in range(10):
                    graphs.append(nx.barabasi_albert_graph(i,j))
        max_prev_node = 20
    elif args=='grid_big':
        graphs = []
        for i in range(36, 46):
            for j in range(36, 46):
                graphs.append(nx.grid_2d_graph(i, j))
        max_prev_node = 90

    elif 'barabasi_noise' in args:
        graphs = []
        for i in range(100,101):
            for j in range(4,5):
                for k in range(500):
                    graphs.append(nx.barabasi_albert_graph(i,j))
        graphs = perturb_new(graphs,p=args.noise/10.0)
        max_prev_node = 99

    # real graphs
    elif args == 'enzymes': 
        graphs= Graph_load_batch(min_num_nodes=10, name='ENZYMES')
        max_prev_node = 25
    elif args == 'enzymes_small':
        graphs_raw = Graph_load_batch(min_num_nodes=10, name='ENZYMES')
        graphs = []
        for G in graphs_raw:
            if G.number_of_nodes()<=20:
                graphs.append(G)
        max_prev_node = 15
    elif args == 'protein':
        graphs = Graph_load_batch(min_num_nodes=20, name='PROTEINS_full')
        max_prev_no.de = 80
    elif args == 'DD':
        graphs = Graph_load_batch(min_num_nodes=100, max_num_nodes=500, name='DD',node_attributes=False,graph_labels=True)
        max_prev_node = 230
    elif args == 'citeseer':
        _, _, G = Graph_load(dataset='citeseer')
        G = max(nx.connected_component_subgraphs(G), key=len)
        G = nx.convert_node_labels_to_integers(G)
        graphs = []
        for i in range(G.number_of_nodes()):
            G_ego = nx.ego_graph(G, i, radius=3)
            if G_ego.number_of_nodes() >= 50 and (G_ego.number_of_nodes() <= 400):
                graphs.append(G_ego)
        max_prev_node = 250
    elif args == 'citeseer_small':
        _, _, G = Graph_load(dataset='citeseer')
        G = max(nx.connected_component_subgraphs(G), key=len)
        G = nx.convert_node_labels_to_integers(G)
        graphs = []
        for i in range(G.number_of_nodes()):
            G_ego = nx.ego_graph(G, i, radius=1)
            if (G_ego.number_of_nodes() >= 4) and (G_ego.number_of_nodes() <= 20):
                graphs.append(G_ego)
        shuffle(graphs)
        graphs = graphs[0:200]
        max_prev_node = 15

    return graphs




In [61]:
graphs = create("ladder")

We currently have no order $ \pi $, nor is the graph in vector form. Using a breadth-first search of the neighbours we get a
indeterminate numbering of the nodes. 
The exact approach is:
1. we choose a random node of the graph to which the node order assigns the number $1$.
2. we choose the neighbouring nodes of this node and $\pi $ assigns a different number to each of them.
3. as in a breadth-first search, take the neighbours of these nodes and assign a number to them.

We stop when the graph is traversed. Thus we have given a node order $ \pi $.

We want to transform the edges into a special vector set. Let $ \pi $ be given. We define $ S_i \in \{ 0, 1 \}^{i - 1} $ in that the jth entry is $ 1 $ if $ \pi(v_i) $ and $ \pi(v_j) $ have an edge, otherwise $ 0 $.

In [ ]:
## changes NX into SI - Arrays
def get_si(graph):
    bfs_list = list()
    q = Queue(maxsize = 0)
    graph_list = list(graph.nodes())
    start_element = random.choice(graph_list)
    q.put(start_element)

    while(q.qsize() > 0):
        n = q.get()
        if not n in bfs_list:
            bfs_list.append(n)
            neighs = list(nx.all_neighbors(graph, n))
            for entry in neighs:
                    q.put(entry)

    result_si = list()
    for i_list, i_node in enumerate(bfs_list[1:]):
        neighs = list(nx.all_neighbors(graph, i_node))
        si = list()

        for i_vector_list, i_vector_node in enumerate(bfs_list[:i_list + 1]):
            if i_vector_node in neighs:
                si.append(1)
            else:
                si.append(0)
        si_np = np.asarray(si)
        result_si.append(si_np)
    return result_si

![title](arch_graphrnn.png)

The neural network actually consists of the two networks $ f_{\text{trans}} $ and $ f_{\text{out}} $. $ f_{\text{trans}} $ is a gated recurrent unit, GRU for short, which has as input $ S_{i - 1} $ and the hidden vector $ h_{i - 1} $ and outputs a new vector $ h_i $, i.e. $ h_i = f_{\text{trans}} (S_{i - 1} , h_{i - 1}) $.

$ f_{\text{out}} $ then takes the hidden vector $h_i $ and creates an $ S_i $.
$ f_{\text{trans}} $ is considered as an edge network and $ f_{\text{out}} $ as a node network.

In (You et al. 2018), two variants are presented: Once, $ f_{\text{out}} $ can be a simple multi-layer perceptron (as implemented here). This variant is called GraphRNN - S in (You et al. 2018). A better variant is to take a GRU for $ f_{\text{out}} $. This variant is then called GraphRNN in (You et al. 2018). RNN here means a Recurrent Neural Network.

In the code block below we create the actual model. For the node creation we use a GRU, while the edge creation is created by MLP. The hidden vector has the 
size 128 and max_nodes specifies the maximum size of all $ S_i $.

SOS (short for start of sequence) and EOS (short for end of sequence) are predefined vectors.

SOS indicates to the network that a new graph is to be generated and EOS indicates when the graph is finished being generated. We use the EOS later to indicate that the graph is ready.

The flow in the **training process** is as follows. We have a list of graphs, where the graphs are 
again consist of ordered lists of $ S_i $ vectors. We take a graph and iterate over an $i$-th $ S_i $, 
starting with $ S_0 $.
At the beginning of each graph, we give SOS as input into $ f_{\text{trans}}$. We take the resulting hidden vector as input
for $ f_{\text{out}} $ and get a vector. The loss function is binary_cross_entropy and takes as argument this vector and $ S_0 $.
Then we apply the gradient descent.
In the next step, we take as input $ S_0 $ and the previous hidden vector for $ f_{\text{trans}}$.
We take the output for $ f_{\text{out}} $ and calculate the loss again with $ S_1 $.

As long as we do not reach the EOS, we take $ S_i $ and the previous hidden vector and calculate the loss with the output and $ S_{i + 1} $.
If $ S_{i + 1} $ is the EOS, we end the loop and start again with a new graph.

The flow in the **generation process** is as follows.
We give SOS as input in $ f_{\text{trans}}$. We take the resulting hidden vector as input
for $ f_{\text{out}} $ and get a vector. We take this vector as a parameter for a binomial random function and get a vector that should be $ S_0 $.
In the next step, we take as input $ S_0 $ and the previous hidden vector for $ f_{\text{trans}}$.
We take the output for $ f_{\text{out}} $ and get a vector again. With this vector we apply a binomial random function
and get $ S_1 $.

As long as we do not get the EOS back, we take $ S_i $ and the previous hidden vector and calculate $ S_{i + 1} $.
If the $ S_{i + 1} $ is the EOS, we end the loop and have a finished graph.

In [5]:
## model
class graphrnn_simple(nn.Module):
    def __init__(self, max_nodes):
        super(graphrnn_simple, self).__init__()
        self.node_network = nn.GRUCell(128 + max_nodes, 128)
        
        
        self.edge_mlp = nn.Linear(128, 500)
        self.intern_act = nn.ReLU()
        self.edge_mlp2 = nn.Linear(500, max_nodes)
        self.act = nn.Sigmoid() 
        torch.nn.init.xavier_uniform_(self.node_network.weight_ih)
        torch.nn.init.xavier_uniform_(self.node_network.weight_hh)
        torch.nn.init.xavier_uniform_(self.edge_mlp.weight)
        torch.nn.init.xavier_uniform_(self.edge_mlp2.weight)
        
    
    def forward(self, input):
        x = self.node_network(input)
        self.hidden_vec = x.clone().detach()
        x = self.edge_mlp(x)
        x = self.intern_act(x)
        x = self.edge_mlp2(x)
        
        x = self.act(x)
        return x

In [6]:
## SOS and EOS
def get_startvector(len_end): #hidden vec only zeros
    start = torch.cat(
        (torch.rand(128), torch.ones(len_end - 128))).view(1,len_end)
    return start


def get_endvector(len_end):
    start = torch.zeros(len_end).view(1,len_end)
    return start.numpy()

In [7]:
## Iterator. Y is X of next iteration
def get_input_output_set(graph_set):
    for j in range(1, len(graph_set)):
        yield graph_set[j - 1], graph_set[j]

In [8]:
# changes np into tensor
def si_to_tensor(si, length=124):
    si_len = si.shape[0] 
    if si_len == length:
        return torch.from_numpy(si).view(1, length).float()
    si_torch = torch.from_numpy(si)
    si_torch = torch.cat(
        (si_torch, torch.zeros(length - si_len))
    ).view(1, length)
    return si_torch

In [9]:
def get_set (graphs, n_clones):
    return [get_si(random.choice(graphs)) for _ in range(n_clones)]

In [11]:
## adds EOS and SOS
def prepare_set(graph_set, limit): 
    return [[np.ones(limit)] + g + [np.zeros(limit)]
                for g in graph_set if len(g) <= limit]

In [78]:

## generates graph based on model
def create_graphs(model, n_graphs, length=100):
    created_graphs = list()
    no_eos_graphs = list()
    for k in range(100):
        start = get_startvector(length + 128)
        distribution = model(start)
        
        inputV = torch.cat(
            (torch.FloatTensor(model.hidden_vec).view(1, 128), distribution.detach()), dim=1
        )
        i = 0
        hs = list()
        dss = list()
        while(True):
            hs.append(inputV)
            distribution = model(inputV)
            dss.append(torch.bernoulli(distribution))
            inputV = torch.cat(
                (torch.FloatTensor(model.hidden_vec).view(1, 128), distribution.detach()), dim=1
            )
            end = torch.from_numpy(get_endvector(distribution.shape[1]))
            i += 1

            ## Vermeidung von Endlosgenerierung
            if i > n_graphs:
                no_eos_graphs.append(dss)
                break
            if ( torch.round(distribution) == end).all():
                hs.append(inputV)
                if i > 1:
                    created_graphs.append(dss)
                    break
                    
    return created_graphs, no_eos_graphs


In [79]:
##changes bfs graphs into nx
def si_to_nx(graph):
    g = nx.Graph()
    for i, si in enumerate(graph):
        g.add_node(i)
        if i == 0:
            continue
        si_local = si[0, :i].detach().numpy()
        
        ind = np.where(si_local == 1.)[0].tolist()
        
        for connec_ind in ind:
            g.add_edge(connec_ind, i)

    return g

In [80]:
def si_to_nx_1d(graph):
    g = nx.Graph()
    for i, si in enumerate(graph):
        g.add_node(i)
        if i == 0:
            continue
 
        si_local = si[:i]
        
        ind = np.where(si_local == 1.)[0].tolist()
        
        for connec_ind in ind:
            g.add_edge(connec_ind, i)

    return g

In [81]:
## constants
training_split = int(len(graphs) * 0.7)
val_split = int(len(graphs) * 0.85)

N_BFS_CLONES = 1000
N_VAL_CLONES = 100

QUANTILE_LIMIT = int((N_BFS_CLONES + 2 * N_VAL_CLONES) * 0.95)

In [82]:
training_set = get_set(graphs[:training_split], N_BFS_CLONES)
validation_set = get_set(graphs[training_split:val_split], N_VAL_CLONES)
test_set = get_set(graphs[val_split:], N_VAL_CLONES)



In [83]:
## creates quantiles to filter to big graphs
len_set = [len(g) for g in training_set + validation_set + test_set]
# mlp needs fixed sized input so choose a 95 % size.
quantile = sorted(len_set)[QUANTILE_LIMIT]
training_set = prepare_set(training_set, quantile)
validation_set = prepare_set(validation_set, quantile)
test_set = prepare_set(test_set, quantile)

We adopt the validation metrics from You et. al, 2018 and refer there for further details. The implementation of the metrics comes from https://github.com/snap-stanford/GraphRNN/blob/master/eval/stats.py

In [84]:
%run eval/stats.py

In [85]:
model = graphrnn_simple(quantile)

criterion = F.binary_cross_entropy
optimizer = optim.Adam(model.parameters(), lr=0.01)

The training process is such that we take $ S_{ i - 1} $ and the hidden vector and expect $ S_{ i} $ as output.
At the beginning we take SOS and a random vector instead of $ S_{ i - 1} $ and the hidden vector, and at the end we expect an EOS.

In [ ]:
writer = SummaryWriter()
step = 0
last_validation_step = 0
VALIDATE_AFTER = 2000
val_dg = 0
val_cs = 0
interesting_models = list()
cut_training_set = 100
for epoch in range(2):
    min_loss = 2

    for i, g in enumerate(training_set[:cut_training_set]):

    
        running_loss = 0

        for i_one_graph, (si_input, si_output) in enumerate(get_input_output_set(g)):
            tensor_input = si_to_tensor(si_input, length=quantile)
            if i_one_graph == 0:
                inputV = torch.cat(
                (torch.rand(1, 128), tensor_input.detach()), dim=1
                ) 
            else:
                inputV = torch.cat(
                (torch.FloatTensor(model.hidden_vec).view(1, 128), tensor_input.detach()), dim=1
                ) 

            distribution = model(inputV)
            
            tensor_output = si_to_tensor(si_output, length=quantile)
            loss = criterion(distribution, tensor_output)

            loss.backward()
            inputV = torch.cat(
                (torch.FloatTensor(model.hidden_vec).view(1, 128), tensor_output.detach()), dim=1
            ) 

            writer.add_scalar("LOSS", loss.item(), step )
            step += 1
            print(f"graph {i} \t EPOCH: {epoch} \t loss  {loss.item():.5f} \t step: {step}", end="\r")

            if loss.item() < min_loss:
                min_loss = loss.item()
                best_model = copy(model)
        distribution = model(inputV)
        optimizer.step()
        
        model.zero_grad()
        optimizer.zero_grad()
        
        ## validation creates graph for validation
        if step - last_validation_step >= VALIDATE_AFTER:
            last_validation_step = step
            syn_graph, _ = create_graphs(model, 10, length=quantile)
            if len(syn_graph) == 0 or step == 0:
                print("\n synthectic 0 \n")
                writer.add_scalar("Degree Staats", 2, step )
                writer.add_scalar("Clustering Stats", 0, step )
                continue
            nx_generated_graphs = [si_to_nx(g) for g in syn_graph]

            val_dg = degree_stats(nx_generated_graphs, graphs[training_split:val_split])
            val_cs = clustering_stats(nx_generated_graphs, graphs[training_split:val_split])
            
            if val_cs > 0 or val_dg < 2 or len(syn_graph) > 0:
                interesting_models.append(copy(model))
            
            writer.add_scalar("Degree Staats", val_dg, step )
            writer.add_scalar("Clustering Stats", val_cs, step )
            

    
writer.close()

In [87]:
test_graphs, _ = create_graphs(model, 300, length=quantile)
nx_generated_graphs = [si_to_nx(g) for g in test_graphs]

In [ ]:

test_set = graphs[val_split:]
val_dg = degree_stats(nx_generated_graphs, test_set)
val_cs = clustering_stats(nx_generated_graphs, test_set)


In [107]:
torch.save(model, "bestmodels/best_model_tree.pt")

In [34]:
model = torch.load("bestmodels/best_model_tree.pt")

## visualisation

In [40]:
## source: https://github.com/snap-stanford/GraphRNN/blob/master/utils.py
def draw_graph_list(G_list, row, col, fname = 'figures/test', layout='spring', is_single=False,k=1,node_size=55,alpha=1,width=1.3):
    # # draw graph view
    # from pylab import rcParams
    # rcParams['figure.figsize'] = 12,3
    plt.switch_backend('agg')
    for i,G in enumerate(G_list[:row * col]):
        plt.subplot(row,col,i+1)
        plt.subplots_adjust(left=0, bottom=0, right=1, top=1,
                        wspace=0, hspace=0)

        plt.axis("off")
        if layout=='spring':
            pos = nx.spring_layout(G,k=k/np.sqrt(G.number_of_nodes()),iterations=100)
            # pos = nx.spring_layout(G)

        elif layout=='spectral':
            pos = nx.spectral_layout(G)
        # # nx.draw_networkx(G, with_labels=True, node_size=2, width=0.15, font_size = 1.5, node_color=colors,pos=pos)
        # nx.draw_networkx(G, with_labels=False, node_size=1.5, width=0.2, font_size = 1.5, linewidths=0.2, node_color = 'k',pos=pos,alpha=0.2)

        if is_single:
            # node_size default 60, edge_width default 1.5
            nx.draw_networkx_nodes(G, pos, node_size=node_size, node_color='#336699', alpha=1, linewidths=0, font_size=0)
            nx.draw_networkx_edges(G, pos, alpha=alpha, width=width)
        else:
            nx.draw_networkx_nodes(G, pos, node_size=1.5, node_color='#336699',alpha=1, linewidths=0.2)
            nx.draw_networkx_edges(G, pos, alpha=0.3,width=0.2)

        # plt.axis('off')
        # plt.title('Complete Graph of Odd-degree Nodes')
        # plt.show()
    plt.tight_layout()
    plt.show()
    plt.savefig(fname+'.png', dpi=600)
    plt.close()

In [53]:
draw_graph_list(graphs[val_split:], 1, 1, fname="figures/test")

<ipython-input-40-5af41a2067aa>:34: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


## Results


Here I will present the results of the networks. First, the test data is visualised. This is followed by diagrams,
describing the loss curve and the validation metrics Degree Stat and Cluster Stat.
Finally, we visualise the generated graphs.
"Ladder" and "tree" are arguments of the create function. The investigations are based on this graph

### Ladder

We have taken 100 graphs per epoch here.

#### visualisation test data


<img src="figures/ladder_test.png" width="1000">

#### plot Loss

<img src="diagramme/ladder_loss0720.png" width="1000">

#### plot Log Loss

<img src="diagramme/ladder_loss_log.png" width="1000">

#### plot Cluster Stat

<img src="diagramme/ladder_clustering_stats0720.png" width="1000">

#### plot Degree Stat

<img src="diagramme/ladder_degree_stats.png" width="1000">

#### visualisation generated data

<img src="figures/ladder_synth.png" width="1000">

### Tree

#### visualisation test data


<img src="figures/tree_test.png" width="1000">

#### plot Loss

<img src="diagramme/loss_tree.png" width="1000">

#### plot Log Loss

<img src="diagramme/loss_log_tree.png" width="1000">

#### plot Cluster Stat

<img src="diagramme/cs_tree.png" width="1000">

#### plot Degree Stat

<img src="diagramme/ds_tree.png" width="1000">

#### visualisation generated data

<img src="figures/tree_synth.png" width="1000">

## Results of Paper

Different graphs were generated in the paper than in this notebook. Therefore, we do not compare with the paper.

## Discusion

The generation is definitely expandable, as can be seen from the visualisation. For the tree graphs, insufficient graphs are generated, while for ladder graphs
ladder graphs we already achieve good results.
One possibility would be to use the more complex variant instead of GraphRNN simple. Furthermore, it would be useful to train longer with better hardware. The training hardware was a Carbon X1 from 2018 with Intel® Core™ i7-7500U CPU @ 2.70GHz. We used GraphRNN - simple as the implementation alone was very time consuming and simpler model is more trainable with weaker hardware.
for
Furthermore, I had to deal with a number of other problems. I could not train with an arbitrary amount of data, because if a model was too overfitted, it tended to stop sending EOS.
I also found it contradictory that the paper did not distinguish between generation and training.
The paper explained the model with a random process, but did not mention that this was omitted for the training process. I only understood this difference when I looked at the actual implementation.

### Sources

- J. You et al, 2018. GraphRNN: Generating Realistic Graphs with Deep Auto-regressive Models In: ICML 2018
